In [1]:
## importing necessary lbraries 
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px  
import plotly.graph_objects as go  
import plotly.io as io  
io.templates.default = 'plotly_dark'
from scipy import stats
from statsmodels.formula.api import ols         
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd 
from statsmodels.stats.multitest import multipletests
import patsy
np.random.seed(10)


creating the data

In [2]:
## creating the dependent variable  
Treatment = np.repeat(['A','B','C'],10)
## the dependent varaiable for each group  
depA =  np.round(stats.norm.rvs(loc=100,scale=10,size=10),1)
## depent for B  
depB = np.round(stats.norm.rvs(loc=105,scale=7,size=10),1)
## the dependent for C 
depC = np.round(stats.norm.rvs(loc=95 , scale= 5,size = 10),1) 


creating the data frame

In [3]:
## create a tabular representation  
df = pd.DataFrame({ 
    'Treatment':Treatment,  
    'Dependent':np.array([depA,depB,depC]).flatten()
})

In [4]:
## displaying the first five itemns of the dataframe  
df.head()

,Treatment,Dependent
0,A,113.3
1,A,107.2
2,A,84.5
3,A,99.9
4,A,106.2


In [5]:
## checking the shape 
df.shape

(30, 2)

In [6]:
## letting pandas know that we have a categorical column  
df.Treatment = pd.Categorical(df.Treatment , categories=['A','B','C'])

In [7]:
## checking the dtype  
df.dtypes

Treatment    category
Dependent     float64
dtype: object

Exploratory Data Analysis

In [8]:
## grouped summary statistics of the dependent variable  
df.groupby('Treatment',observed=False).Dependent.describe()

,count,mean,std,min,25%,50%,75%,max
Treatment,,,,,,,,
A,10.0,100.60,7.937674,84.5,98.700,100.55,105.325,113.3
B,10.0,106.22,6.713635,97.0,100.125,107.30,111.175,115.4
C,10.0,96.78,7.055778,85.1,94.075,97.20,101.650,106.9


* visualize the data

In [9]:
px.box( 
    df, 
    x = 'Treatment', 
    y = 'Dependent', 
    color='Treatment',
    title= 'Dependent Variable of the Treatment groups'
)

* Line of best fit is the mean of the dependent variable for each treatment level

In [10]:
px.scatter( 
    df, 
    x = 'Treatment', 
    y = 'Dependent'
 
)

In [11]:
df[:5]

,Treatment,Dependent
0,A,113.3
1,A,107.2
2,A,84.5
3,A,99.9
4,A,106.2


Linear Model
- `ols`

In [12]:
linear_Model =  ols('Dependent ~ Treatment',data=df).fit()
## check out the resutlts  
linear_Model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Dependent   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     4.285
Date:                Thu, 23 Jan 2025   Prob (F-statistic):             0.0242
Time:                        14:22:36   Log-Likelihood:                -100.43
No. Observations:                  30   AIC:                             206.9
Df Residuals:                      27   BIC:                             211.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        100.6000      2.294     43.855      0.000      95.893     105.307
Treatment[T.B]     5.6200      3.244      1.732      0.095      -1.036      12.276
Treatment[T.C]    -3.8200      3.244     -1.178      0.249     -10.476       2.836
==============================================================================
Omnibus:                        1.137   Durbin-Watson:                   1.902
Prob(Omnibus):                  0.566   Jarque-Bera (JB):                1.079
Skew:                          -0.410   Prob(JB):                        0.583
Kurtosis:                       2.563   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* The ANOVA TABLE

In [13]:
## Tthe anova table  
anova_lm(linear_Model)

,df,sum_sq,mean_sq,F,PR(>F)
Treatment,2.0,450.968,225.484000,4.285042,0.024197
Residual,27.0,1420.772,52.621185,NaN,NaN


* Understanding the results
* - Interests is in the coefficints of the two levels beta 1 and beta 2 

In [14]:
## coefficeints  
linear_Model.params

Intercept         100.60
Treatment[T.B]      5.62
Treatment[T.C]     -3.82
dtype: float64

In [15]:
## creating the ddesign matix  
y , X = patsy.dmatrices('Dependent~Treatment',df)

In [16]:
## x  
X[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [17]:
## y  
y[:5]

array([[113.3],
       [107.2],
       [ 84.5],
       [ 99.9],
       [106.2]])

In [18]:
## coverting to numy arrays  
y = np.array(y)
X = np.array(X)

In [19]:
XT = X.transpose()
XTX = np.matmul(XT, X)
XTXI = np.linalg.inv(XTX)
XTXIXT = np.matmul(XTXI,XT)
beta = np.matmul(XTXIXT , y)

In [20]:
## print beta  
beta

array([[100.6 ],
       [  5.62],
       [ -3.82]])

In [21]:
## saving the beta into variables  
beta0 = beta[0]
beta1 = beta[1]
beta2 = beta[2]
beta0, beta1 , beta2

(array([100.6]), array([5.62]), array([-3.82]))

In [22]:
## creating a dunction c 
def research(level):  
    if level == 'A':  
        estimate = beta0  
    elif level == 'B': 
        estimate = beta0 + beta1  
    else: 
        estimate = beta0 + beta2 

    return estimate

In [23]:
## The estimate for A 
research('A')

array([100.6])

In [24]:
## the same as doing  
df.loc[df.Treatment == 'A', 'Dependent'].mean()

100.6

In [25]:
## The estimate for B c
research('B')

array([106.22])

In [26]:
## its the same as doing  
df.loc[df.Treatment == 'A','Dependent'].mean() 
 

100.6

In [27]:
## the stimate for C  
research('C')

array([96.78])

In [28]:
## same as doing    
df.loc[df.Treatment== 'C','Dependent'].mean()

96.78

* stanard error of the estimate

In [29]:
linear_Model.bse

Intercept         2.293931
Treatment[T.B]    3.244108
Treatment[T.C]    3.244108
dtype: float64

In [30]:
##saving the standard errors into computer variables  
se0 = linear_Model.bse.iloc[0]
se1 = linear_Model.bse.iloc[1]
se2  = linear_Model.bse.iloc[2]

In [31]:
## the test T statsistic  
linear_Model.tvalues


Intercept         43.854854
Treatment[T.B]     1.732371
Treatment[T.C]    -1.177519
dtype: float64

In [32]:
## Test statistcixc for coefficient B  
test_statistc_for_beta1 = beta1 / se1 
test_statistc_for_beta1

array([1.7323714])

In [33]:
## test statisct for beta c
test_stats_c = beta2/se2 
test_stats_c

array([-1.17751935])

* pvalues

In [34]:
linear_Model.pvalues ## from the t - distribution

Intercept         1.236666e-26
Treatment[T.B]    9.461814e-02
Treatment[T.C]    2.492639e-01
dtype: float64

In [35]:
## confidentce intervals  
linear_Model.conf_int()

,0,1
Intercept,95.893243,105.306757
Treatment[T.B],-1.036360,12.276360
Treatment[T.C],-10.476360,2.836360


In [36]:
## Returning to the Model 
anova_lm(linear_Model)

,df,sum_sq,mean_sq,F,PR(>F)
Treatment,2.0,450.968,225.484000,4.285042,0.024197
Residual,27.0,1420.772,52.621185,NaN,NaN


In [37]:
## first five fitted values  
linear_Model.fittedvalues[:5]

0    100.6
1    100.6
2    100.6
3    100.6
4    100.6
dtype: float64

In [38]:
## asigning them to the dataframe  
df['fittedvalues'] = linear_Model.fittedvalues

In [39]:
## checking the dataframe   
df.head()

,Treatment,Dependent,fittedvalues
0,A,113.3,100.6
1,A,107.2,100.6
2,A,84.5,100.6
3,A,99.9,100.6
4,A,106.2,100.6


In [40]:
## sum of squares due to the regression  
linear_Model.ess

450.96800000000053

In [41]:
SSE = np.sum((df.fittedvalues - df.Dependent.mean())**2)
SSE

450.9679999999999

In [42]:
## sum of squares due to the error  
linear_Model.ssr

1420.7720000000002

In [43]:
## calcualting the using hand  
SSR = np.sum((df.Dependent - df.fittedvalues)**2)
SSR

1420.7720000000002

In [44]:
## SST   
sst = SSE + SSE 
sst


901.9359999999998

* Degrees of freedom for the numerator and the denominator
   - Num of observations
   - Num of parameters

In [45]:
## Number of observations  
m = df.shape[0]
m

30

In [46]:
## Number of parameters  
k = 3
k

3

In [47]:
## degrees of freedom for the model  
linear_Model.df_model

2.0

In [48]:
## degrees of freedom for the residual  
linear_Model.df_resid

27.0

In [49]:
## Numeratoe degrees of freddom 
df_1  = k - 1
df_1

2

In [50]:
## Denominator degrees of freedom     
df_2 = m - k  
df_2

27

In [51]:
## mean squared error due the resid  
linear_Model.mse_model

225.48400000000026

In [52]:
## mean squared error due to the residdual 
linear_Model.mse_resid

52.62118518518519

In [53]:
## fvalues  
f = linear_Model.fvalue
f

4.2850422164851265

In [54]:
linear_Model.f_pvalue

0.024197177992641736

In [55]:
## the ep value using cummualative disyribution 
1 - stats.f.cdf(f , df_1,df_2)

0.02419717799264176

* Visualize the F -  Distribution , critical value and the F-stastic

In [56]:
xvals = np.linspace(stats.f.ppf(0.01 , df_1 ,df_2),stats.f.ppf(0.99,df_1 ,df_2),200)
f_critic = stats.f.ppf(0.95, df_1 , df_2)
## creating the figure  
fig = go.Figure()
fig.add_trace( 
    go.Scatter( 
        x = xvals, 
       y = stats.f.pdf(xvals,df_1 , df_2), 
       mode= 'lines', 
       name= 'PDF' 
    )
)

fig.add_trace( 
    go.Scatter( 
        x = [f_critic], 
        y = [0], 
        mode ='markers', 
        marker= dict(color = 'red', size = 20), 
        name = 'Critical_F'

    )
)
fig.add_trace( 
    go.Scatter( 

        x = [f], 
        y = [0], 
        mode = 'markers', 
        marker= dict(color='green',size=20), 
        name ='F-Statsisc'
    )
)
fig.update_layout(title='PDF',xaxis=dict(title = 'F values'))

In [57]:
## coeffficeint of determination 
linear_Model.rsquared

0.2409351726201291

* Tukeys Honstely Significant Test

In [60]:
## pairwise  
mpc = pairwise_tukeyhsd(df.Dependent , df.Treatment,alpha=0.05)
mpc.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
A,B,5.62,0.2118,-2.4235,13.6635,False
A,C,-3.82,0.4764,-11.8635,4.2235,False
B,C,-9.44,0.019,-17.4835,-1.3965,True


In [62]:
groupA = df.loc[df.Treatment=='A','Dependent'].to_list()
groupB = df.loc[df.Treatment=='B','Dependent'].to_list()
groupC = df.loc[df.Treatment=='C','Dependent'].to_list()

In [70]:
## t tets for A vs B  
ttetAB = stats.ttest_ind(groupA , groupB)
##accessing the pvalues  
ttetAB.pvalue

0.10454491882831159

In [69]:
## ttest for A and C  
ttestAC = stats.ttest_ind(groupA , groupC)
ttestAC.pvalue

0.27027134187109586

In [67]:
## test for B and C  
ttestBC = stats.ttest_ind(groupB , groupC)
ttestBC.pvalue

0.006669762292835393

In [72]:
multipletests([ttetAB.pvalue,ttestAC.pvalue,ttestBC.pvalue],alpha=0.05,method='bonferroni')

(array([False, False,  True]),
 array([0.31363476, 0.81081403, 0.02000929]),
 0.016952427508441503,
 0.016666666666666666)